In [6]:
# imports
# from astropy.io import fits
# import matplotlib.pyplot as plt
import numpy as np
import csv 
# import match

In [2]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Get AGN IDs

In [9]:
# initialize list
agn_file = []

# open csv file: D:\IfA REU\IfA-REU-Project\Lists\Match_COSMOS2020_IRAGN_to_Chandra2016_XrayAGN.csv
with open(path_csv+'\Match_COSMOS2020_IRAGN_to_Chandra2016_XrayAGN.csv', newline='') as csv_file : 
    reader = csv.reader(csv_file, delimiter=',', quotechar='"')
    # output file into list for easy access
    for row in reader :
        agn_file.append(row)

# print column names
print(agn_file[0])

['ID', 'i COSMOS2020', ' i CHANDRA 2016']


In [19]:
# cut out heading names and transpose to access columns
agn_id, agn_i20, agn_i16 = np.array(agn_file[1:], dtype=int).T

# test: left and right must match
print(agn_file[1][0], '==', agn_id[0])
print(agn_file[1][1], '==', agn_i20[0])
print(agn_file[1][2], '==', agn_i16[0])

100728 == 100728
61879 == 61879
1866 == 1866


# Get Data From Catalogs